# Generamos Foto Reservas desde Histórico

_Importación de librerias_

In [1]:
import pandas as pd
import numpy as np
import tfm_helpers as tfm

pd.set_option("display.max_columns", None)

Cargamos el histórico de reservas

In [2]:
file = tfm.getPathToCleanedHistoricalReservasHotel("parquet")
print(file)
historico_reservas = pd.read_parquet(file)
historico_reservas.shape
historico_reservas.head()


data\cleaned\historical\reservas\parquet\historico_reservas.parquet


,Hotel,Fecha venta,Cliente,TTOO,Agencia,Fecha entrada,Fecha salida,Noches,Nacionalidad,Régimen factura,Cantidad habitaciones,AD,JR,NI,CU,Tipo facturación estancia,Estado reserva,Desglosada,Usuario venta,Fecha Borrado,Usuario Borrado,Day Use,Grupo,Master del grupo,Canal_Origen,Segmento,Régimen uso,Canal_Origen Aux,Id Tipo habitacion uso,Id Tipo habitacion factura,Días Anticipación
Id Reserva,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
02-2013-2784-1,02,2013-04-17,9028,DIRECTOS,DIRECTOS,2013-04-17,2013-04-18,1,None,AL,1,1,0,0,0,Directo,Anulada,1,TOMEUSC,2013-04-17,02JEFEREC,0,None,0,None,None,AL,None,02-A,02-A,0
02-2013-2807-1,02,2013-04-19,367,CUENTA CASA,CUENTA CASA,2013-04-20,2013-04-21,1,ES,AI,1,2,0,1,0,Directo,Anulada,1,ZENNIS,2013-04-20,MARIAMBP,0,None,0,None,None,AI,None,02-RS,02-RS,1
02-2013-2565-1,02,2013-04-08,367,CUENTA CASA,CUENTA CASA,2013-04-19,2013-04-21,2,ES,AI,1,2,2,0,0,Directo,Salida,1,02JEFEREC,1753-01-01,None,0,None,0,None,None,AI,None,02-RS,02-A,11
02-2013-2596-1,02,2013-04-11,9028,DIRECTOS,DIRECTOS,2013-04-19,2013-04-21,2,None,AI,1,2,0,1,0,Directo,Salida,1,BARBARAMRM,1753-01-01,None,0,None,0,None,None,AI,None,02-RS,02-A,8
02-2013-2646-1,02,2013-04-13,5041,HOTELS4U.COM,JUMBO TOURS,2013-04-19,2013-04-22,3,None,MPP,1,2,1,0,0,Agencia,Salida,1,NAS,1753-01-01,None,0,None,0,None,None,MPP,None,02-SCA,02-A,6


In [3]:
historico_reservas.dtypes

Hotel                                 object
Fecha venta                   datetime64[ns]
Cliente                               object
TTOO                                  object
Agencia                               object
Fecha entrada                 datetime64[ns]
Fecha salida                  datetime64[ns]
Noches                                 int32
Nacionalidad                          object
Régimen factura                       object
Cantidad habitaciones                  int32
AD                                     int64
JR                                     int64
NI                                     int64
CU                                     int64
Tipo facturación estancia             object
Estado reserva                        object
Desglosada                             int32
Usuario venta                         object
Fecha Borrado                 datetime64[ns]
Usuario Borrado                       object
Day Use                                int32
Grupo     

In [4]:
historico_reservas["Fecha venta"].describe(datetime_is_numeric=True)

count                           486349
mean     2015-07-09 23:42:55.313612288
min                2005-07-26 00:00:00
25%                2013-03-17 00:00:00
50%                2016-04-18 00:00:00
75%                2018-06-07 00:00:00
max                2022-01-05 00:00:00
Name: Fecha venta, dtype: object

In [5]:
fecha_venta_inicial = historico_reservas["Fecha venta"].min()
fecha_venta_final = historico_reservas["Fecha venta"].max()

print(f"La primera fecha de venta es {fecha_venta_inicial} y la última {fecha_venta_final}")

La primera fecha de venta es 2005-07-26 00:00:00 y la última 2022-01-05 00:00:00


In [6]:
historico_reservas["Estado reserva"].unique()

array(['Anulada', 'Salida', 'No Show'], dtype=object)

In [9]:
def create_bookings_snapshot(historico, fecha_foto):
        current_foto_reservas = pd.DataFrame()
        current_fecha = fecha_foto

        filtro_foto_reservas = (historico["Fecha venta"] <= current_fecha) & (historico["Fecha salida"] > current_fecha)
        current_foto_reservas = historico.loc[filtro_foto_reservas].copy(deep=True)
        if not current_foto_reservas.empty:
                current_foto_reservas.loc[:, "Fecha foto"] = current_fecha

                filtro_estado_reserva = (current_fecha <= current_foto_reservas["Fecha entrada"]) & ((current_foto_reservas["Fecha Borrado"] == "1753-01-01") | (current_foto_reservas["Fecha Borrado"] > current_fecha))
                filtro_estado_entrada = (current_fecha > current_foto_reservas["Fecha entrada"]) & (current_foto_reservas["Fecha Borrado"] == "1753-01-01") & (current_foto_reservas["Fecha salida"] > current_fecha)
                filtro_estado_salida = (current_foto_reservas["Fecha salida"] <= current_fecha) & (current_foto_reservas["Fecha Borrado"] == "1753-01-01")
                filtro_estado_anulada = (current_foto_reservas["Fecha Borrado"] < current_fecha) & (current_foto_reservas["Fecha Borrado"] != "1753-01-01")
                filtro_estado_noshow = ((current_foto_reservas["Fecha Borrado"] == current_foto_reservas["Fecha entrada"]) & (current_foto_reservas["Fecha Borrado"] <= current_fecha))

                current_foto_reservas["Estado reserva"] = np.where(filtro_estado_reserva, 
                                                                "Reserva", 
                                                                np.where(filtro_estado_entrada, 
                                                                        "Entrada",
                                                                        np.where(filtro_estado_salida, 
                                                                                "Salida",
                                                                                np.where(filtro_estado_anulada,
                                                                                        "Anulada",
                                                                                        np.where(filtro_estado_noshow,
                                                                                                "No Show",
                                                                                                current_foto_reservas["Estado reserva"]
                                                                                                )
                                                                                        )
                                                                                )
                                                                        )
                                                                )
        return current_foto_reservas

In [10]:
fotos_reservas = pd.DataFrame()
for fecha in pd.date_range(fecha_venta_inicial, fecha_venta_final):
    current_foto = create_bookings_snapshot(historico_reservas, fecha)
    if not current_foto.empty:
        # current_foto.to_csv(tfm.getPathToProcessedSnapshotReservas("csv", fecha), index=True, sep="|", escapechar="~", encoding="cp1252")
        current_foto.to_parquet(tfm.getPathToProcessedSnapshotReservas("parquet", fecha), index=True)
        fotos_reservas = current_foto

fotos_reservas.head()

(5, 32)
(0, 31)
(0, 31)
(0, 31)
(0, 31)
(0, 31)
(1, 32)
(7, 32)


,Hotel,Fecha venta,Cliente,TTOO,Agencia,Fecha entrada,Fecha salida,Noches,Nacionalidad,Régimen factura,Cantidad habitaciones,AD,JR,NI,CU,Tipo facturación estancia,Estado reserva,Desglosada,Usuario venta,Fecha Borrado,Usuario Borrado,Day Use,Grupo,Master del grupo,Canal_Origen,Segmento,Régimen uso,Canal_Origen Aux,Id Tipo habitacion uso,Id Tipo habitacion factura,Días Anticipación,Fecha foto
Id Reserva,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16-2022-1854-25,16,2022-01-05,5544,TEAM INEOS,TEAM INEOS,2022-01-08,2022-01-10,2,GB,PCP,1,1,0,0,0,Agencia,Reserva,1,VIVA\JEFEREC16,1753-01-01,None,0,None,0,None,C,PCP,None,16-PRIA,16-A,3,2022-01-05
16-2022-1853-24,16,2022-01-05,5544,TEAM INEOS,TEAM INEOS,2022-01-07,2022-01-11,4,GB,PCP,1,1,0,0,0,Agencia,Reserva,1,VIVA\JEFEREC16,1753-01-01,None,0,None,0,None,C,PCP,None,16-PRIA,16-A,2,2022-01-05
16-2022-1854-28,16,2022-01-05,5544,TEAM INEOS,TEAM INEOS,2022-01-10,2022-01-12,2,GB,PCP,1,1,0,0,0,Agencia,Reserva,1,VIVA\BARTOLOMECB,2022-01-10,VIVA\JEFEREC16,0,None,0,None,C,PCP,None,16-A,16-A,5,2022-01-05
16-2022-1853-1,16,2022-01-05,5544,TEAM INEOS,TEAM INEOS,2022-01-07,2022-01-12,5,GB,PCP,1,1,0,0,0,Agencia,Reserva,1,VIVA\JEFEREC16,1753-01-01,None,0,None,0,None,C,PCP,None,16-A,16-A,2,2022-01-05
16-2022-1853-10,16,2022-01-05,5544,TEAM INEOS,TEAM INEOS,2022-01-07,2022-01-12,5,ES,PCP,1,1,0,0,0,Agencia,Reserva,1,VIVA\JEFEREC16,1753-01-01,None,0,None,0,None,C,PCP,None,16-A,16-A,2,2022-01-05
